# ROTAS test 

Kerry Wood (kerry.wood@asterism.ai)

In [1]:
import pandas as pd
import ctypes
import load_utils
import helpers

In [2]:
# this will only work if you have the full astrostandards
load_utils.init_all()

EnvConstDll: 0
TimeFuncDll: 0
AstroFuncDll: 0
TleDll: 0
SpVecDll: 0
VcmDll: 0
ExtEphemDll: 0
Sgp4PropDll: 0
SpPropDll: 0
ElOpsDll: 0
SatStateDll: 0
SensorDll: 0
ObsDll: 0
ObsOpsDll: 0
LamodDll: 0
RotasDll: 0
BatchDCDll: 0
b''


0

In [50]:
obs = pd.read_json('./48340_obs.json')
tle = pd.read_json('./48340_tle.json')
# load the obs
obs = obs[ ~obs['idSensor'].isnull() ].copy()
obs = obs[ ~obs['id'].isnull() ].copy()
# this is a data holder / formatter
OBSHELPER = helpers.astrostd_named_fields( load_utils.ObsDll, prefix='XA_OBS_' )

EFG = (ctypes.c_double * 3)()
def toEFG( R ):
    LLH = load_utils.AstroFuncDll.LLHToEFGPos( 
                                              (ctypes.c_double * 3)( R['senlat'],R['senlon'],R['senalt']),
                                              EFG
                                              )
    return list(EFG)

obs['efg'] = obs.apply( toEFG, axis=1 ) 
obs['obTime'] = pd.to_datetime( obs['obTime'] )
obs['ds50_utc'] = obs['obTime'].apply( lambda X : helpers.datetime_to_ds50(X,load_utils.TimeFuncDll) )

In [51]:
def addOb( R, fakeSatNo=777 ):
    satno = R['satNo']
    senx, seny, senz = R['efg']
    # data holders
    OBSHELPER.clear()
    OBSHELPER['XA_OBS_SECCLASS'] = 1
    OBSHELPER['XA_OBS_SATNUM']   = satno
    OBSHELPER['XA_OBS_SITETAG']  = satno
    OBSHELPER['XA_OBS_SPADOCTAG']  = satno
    OBSHELPER['XA_OBS_SENNUM']   = fakeSatNo
    OBSHELPER['XA_OBS_DS50UTC']  = R['ds50_utc']
    OBSHELPER['XA_OBS_ELORDEC']  = R['declination']
    OBSHELPER['XA_OBS_AZORRA']   = R['ra']
    OBSHELPER['XA_OBS_POSX']     = senx
    OBSHELPER['XA_OBS_POSY']     = seny
    OBSHELPER['XA_OBS_POSZ']     = senz
    OBSHELPER['XA_OBS_OBSTYPE']  = 9
    OBSHELPER['XA_OBS_TRACKIND'] = 3 # start of track
    OBSHELPER['XA_OBS_YROFEQNX'] = 2 # J2K equinox
    R['asObId']                 = load_utils.ObsDll.ObsAddFrArray( OBSHELPER.getData() )
    return R['asObId']

# add the obs to the astrostandards
load_utils.ObsDll.ObsRemoveAll()
obs['asObId'] = obs.apply( addOb, axis=1 )

In [52]:
# init the sensor (fake sat number must be same as above)
load_utils.SensorDll.SensorSetLocAll( 777, 0, 0, (ctypes.c_double * 3)(0,0,0), load_utils.Cstr('TEST',24), 99999, 0 )

0

In [53]:
this_tle = tle.iloc[0]
load_utils.TleDll.TleRemoveAllSats()
as_tle   = load_utils.TleDll.TleAddSatFrLines( 
    load_utils.Cstr( this_tle['line1'], 512 ),
    load_utils.Cstr( this_tle['line2'], 512 ) )
assert 0 == load_utils.SatStateDll.SatStateInitSat( as_tle )

In [54]:
# data holders
satelt = (ctypes.c_double * 9)()
obelt  = (ctypes.c_double * 9)()
b3s    = load_utils.Cstr('',512)
# get a ROTAS holder
OBSRESHELPER = helpers.astrostd_named_fields( load_utils.RotasDll, prefix='XA_OBSRES_' )

# assume that TLE is loaded and the sensor is setup...
def ROTASob( R ):
    assert 0 == load_utils.RotasDll.RotasComputeObsResiduals( R['asObId'], as_tle, OBSRESHELPER.data, satelt, obelt )
    return OBSRESHELPER.toDict()

rv = obs.apply( ROTASob, axis=1 ).values
rv = pd.DataFrame.from_records(rv)
total = pd.concat( (obs, rv), axis=1 )

# OpenROTAS test

In [55]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.path.abspath(''), '..')))
from datetime import datetime,timedelta,timezone
import pandas as pd

from astrostandards.utils import load_utils as harness
from OpenROTAS import time_helpers
from OpenROTAS import sgp4_prop
from OpenROTAS import sensor_helper
from OpenROTAS import compute_looks

In [56]:
obs['obTime'].apply( lambda X : helpers.datetime_to_ds50(X,load_utils.TimeFuncDll) )

1      27514.183370
2      27514.183357
3      27514.183335
4      27514.183322
5      27514.183310
           ...     
126    27514.181806
127    27514.181796
128    27514.181783
129    27514.181771
130    27514.181761
Name: obTime, Length: 130, dtype: float64

In [59]:
import importlib
importlib.reload( time_helpers )

X = total['obTime'].iloc[0]
time_helpers.convert_times( total['obTime'], harness )
# total['obTime'].values

2025-04-30 04:24:03.127392+00:00
2025-04-30 04:24:02.053440+00:00
2025-04-30 04:24:00.121536+00:00
2025-04-30 04:23:59.047583+00:00
2025-04-30 04:23:57.973632+00:00
2025-04-30 04:23:57.114816+00:00
2025-04-30 04:23:56.040000+00:00
2025-04-30 04:23:54.967776+00:00
2025-04-30 04:23:54.107232+00:00
2025-04-30 04:23:53.034144+00:00
2025-04-30 04:23:52.175328+00:00
2025-04-30 04:23:51.101376+00:00
2025-04-30 04:23:50.027424+00:00
2025-04-30 04:23:49.168608+00:00
2025-04-30 04:23:48.094656+00:00
2025-04-30 04:23:47.020704+00:00
2025-04-30 04:23:46.161024+00:00
2025-04-30 04:23:45.087072+00:00
2025-04-30 04:23:44.014848+00:00
2025-04-30 04:23:43.155168+00:00
2025-04-30 04:23:42.081215+00:00
2025-04-30 04:23:41.007264+00:00
2025-04-30 04:23:40.148448+00:00
2025-04-30 04:23:39.074496+00:00
2025-04-30 04:23:38.000544+00:00
2025-04-30 04:23:37.141728+00:00
2025-04-30 04:23:36.067776+00:00
2025-04-30 04:23:34.994688+00:00
2025-04-30 04:23:34.135008+00:00
2025-04-30 04:23:33.061056+00:00
2025-04-30

ArgumentError: argument 1: TypeError: wrong type